# Tools for curiosity

In [59]:
import pandas as pd
import numpy as np
import plotly.express as px
import openai
from openai.embeddings_utils import get_embedding
from sklearn.manifold import TSNE
import plotly.io as pio
import umap
pio.renderers.default='notebook'
import textwrap
import plotly.graph_objects as go


with open('../openai_secret_key') as r:
    openai_secret_key = r.readline().strip()
    openai.api_key = openai_secret_key

In [158]:
tools_filename = 'TFC 23-7-19.csv'
tools = pd.read_csv(tools_filename, keep_default_na=False)

tools['prompt_text'] = tools.apply(
    lambda x: x.Name + ': ' + x.Description, axis=1)
tools['prompt_text_tags'] = tools.apply(
    lambda x: x.prompt_text + ' Tags: ' + x.Tags, axis=1)
tools['hover_text'] = tools.apply(
    lambda x: x.Name + ':<br>' + '<br>'.join(textwrap.wrap(x.Description, width=50)), axis=1)

In [162]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"

embeddings = openai.Embedding.create(input=tools.prompt_text_tags.tolist(), model=embedding_model)['data']
tools['embedding'] = [e['embedding'] for e in embeddings]

In [165]:
tools.to_json('tools_embeddings_23-7-20.json')

In [166]:
tools = pd.read_json('tools_embeddings_23-7-20.json')

In [47]:
tags = []
for i, tool in tools.iterrows():
    tags += [t for t in str.split(tool.Tags, ', ') if t not in tags]

In [163]:
embeddings_matrix = np.array(tools.embedding.values.tolist())
fit = umap.UMAP(n_neighbors=35, 
                min_dist=0.01, 
                n_components=2, 
                metric='cosine',
                spread=1,
                random_state=42,
                low_memory=False)
u = fit.fit_transform(embeddings_matrix)
tools['vis_dim1'] = u[:,0]
tools['vis_dim2'] = u[:,1]


/Users/dani/.virtualenvs/research/lib/python3.11/site-packages/umap/umap_.py:2344: UserWarning:

n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1



In [99]:
fig = px.scatter(tools, 
                 x='vis_dim1', y='vis_dim2', 
                 custom_data=['hover_text'],
                 hover_name='hover_text', 
                 template='plotly_dark',
                 hovertemplate='%{customdata[0]}}<extra></extra>'
                )
fig.update_traces(textposition='top center')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.update(layout_coloraxis_showscale=False)
fig.update_layout(showlegend=False)
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.update_traces(hovertemplate="%{customdata[0]}}<extra></extra>")
fig.update_layout(
    font=dict(
        size=11,  # Set the font size here
    )
)
for tag in tags:
    tag_shape = get_tag_shape(tag, tools)
    fig.add_trace(tag_shape)
fig.show(renderer='browser')

TypeError: scatter() got an unexpected keyword argument 'hovertemplate'

# Polygon shapes per tag

In [84]:
tag = tags[0]
tag_tools = tools.loc[tools.Tags.str.contains(tag)]
fig = go.Figure(go.Scatter(x=tag_tools.vis_dim1, y=tag_tools.vis_dim2, 
                           fill="toself", 
                           line={'width': 0},
                           marker={'opacity': 0},
                           #hoverinfo='text',
                           hoveron='fills',
                           hovertemplate='${tag}<extra></extra>',
                           text=tag
                           ))
fig.update_traces(textposition='top center')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.update(layout_coloraxis_showscale=False)
fig.update_layout(showlegend=False)
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.update_layout(
    font=dict(
        size=11,  # Set the font size here
    )
)

fig.show(renderer='browser')

In [141]:
def get_tag_shape(tag, tools):
    tag_tools = tools.loc[tools.Tags.str.contains(tag)]
    return go.Scatter(x=tag_tools.vis_dim1, y=tag_tools.vis_dim2, 
                      fill="toself",
                      opacity=0.1,
                      line={'width': 0},
                      marker={'opacity': 0},
                      name=tag,
                      hoveron='fills',
                     )

In [164]:
traces = []

for tag in []:
    traces.append(get_tag_shape(tag, tools))

scatter = go.Scatter(x=tools.vis_dim1, y=tools.vis_dim2,
                     mode='markers',
                     text=tools.hover_text,
                     customdata=tools.hover_text,
                     hovertemplate='%{customdata}<extra></extra>',
                    )
traces.append(scatter)
fig = go.Figure(data=traces, layout={'template': 'plotly_dark'})

fig.update_traces(textposition='top center')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.update(layout_coloraxis_showscale=False)
fig.update_layout(showlegend=False)
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.update_layout(
    font=dict(
        size=11,  # Set the font size here
    )
)

fig.show(renderer='browser')

In [121]:
scatter = go.Scatter(x=tools.vis_dim1, y=tools.vis_dim2,
                     mode='markers',
                     text=tools.hover_text,
                     customdata=tools.hover_text,
                     hovertemplate='%{customdata}<extra></extra>',
                    )
fig = go.Figure(data=[scatter], layout={'template': 'plotly_dark'})
fig.show(renderer='browser')

# Constellations per tag

In [167]:
tag = tags[0]
tag_tools = tools.loc[tools.Tags.str.contains(tag)]